# Add clone assignments to Seurat object and analyze clonal transcriptional profiles

In [ ]:
library(ggplot2)
library(ggrepel)
library(plyr)
library(dplyr)
library(Seurat)
library(tidyr) 
library(DescTools)

# Color palettes and helper functions
source("../sc_analysis_palettes_and_helpers.r")

dat_name <- "allClones_allTPs"

## Add high-resolution clone assignments to primary tumor data

### Load full Seurat object


In [3]:
# Load full Seurat object

load('../transcriptome_analysis/data_allTPs/all_timepoints_merged_clustered.Robj')


In [4]:
# Subset for MULTI-seq datasets with lineage info and cells belonging to a transplantation experiment

Idents(dat) <- 'orig.ident'
dat <- subset(dat, idents = c('multi_bAllos_1_allos','multi_bAllos_1_tum',
                              'multi_bAllos_2_allos','multi_bAllos_2_tum','multi_bAllos_2_lateAllos',
                              'multi_bAllos_3_allos','multi_bAllos_3_tum',
                              'multi_bAllos_4_allos','multi_bAllos_4_PTs_S1','multi_bAllos_4_PTs_S2',
                              'multi_bAllos_4_lateAllos_S1','multi_bAllos_4_lateAllos_S2','multi_bAllos_4_lateAllos_02_S2')
             )


In [5]:
table(dat$orig.ident)


          multi_bAllos_1_allos             multi_bAllos_1_tum 
                          6966                           7824 
          multi_bAllos_2_allos       multi_bAllos_2_lateAllos 
                          8923                           6395 
            multi_bAllos_2_tum           multi_bAllos_3_allos 
                          3377                           5300 
            multi_bAllos_3_tum           multi_bAllos_4_allos 
                          6590                            861 
multi_bAllos_4_lateAllos_02_S2    multi_bAllos_4_lateAllos_S1 
                          9507                           7066 
   multi_bAllos_4_lateAllos_S2          multi_bAllos_4_PTs_S1 
                          5884                           9916 
         multi_bAllos_4_PTs_S2 
                         10017 

### Merge clone assignments for all allograft datasets

In [6]:
sample_specs <- read.delim('./data/allo_sample_specs.csv', sep = ';', stringsAsFactors = F)
pt_allo_combos <- read.delim('./data/pt_allo_connect.csv',sep = ';', stringsAsFactors = F, header = T)
sample_specs_pt <- read.delim('./data/PT_sample_specs.csv', sep = ';', stringsAsFactors = F)

sample_specs_pt <- sample_specs_pt[sample_specs_pt$dataset_name %in% pt_allo_combos$pt_dataset,]


Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on './data/pt_allo_connect.csv'”


In [127]:
allClones <- data.frame()

#Idents(dat) <- 'dataset'

for(i in 1:nrow(sample_specs_pt)){

    dataset_it <- sample_specs_pt$dataset_name[i]
    print(paste0('Adding dataset ',dataset_it))
    earlyAllo_dataset <- pt_allo_combos[pt_allo_combos$pt_dataset == dataset_it, 'earlyAllo_dataset']
    lateAllo_dataset <-  pt_allo_combos[pt_allo_combos$pt_dataset == dataset_it, 'lateAllo_dataset']    
    
    # Load and process data
    early_dat_i <- read.delim(paste0(sample_specs_pt$sample_wd[i],'/unfiltered_comb_seq_ids_with_PT_assignment_earlyAllos.csv'), sep = ',', stringsAsFactors = F, row.names = 1)
    early_dat_i <- early_dat_i[!duplicated(early_dat_i$bc_clone),]
    early_dat_i <- early_dat_i[,c('Barcode','code_combinations','pt_fish','fish_clone','inferred_clone','assigned_to_pt_fish','assigned_to_pt_clone')]
    early_dat_i$dataset_allo <- earlyAllo_dataset
    
    if(earlyAllo_dataset %in% c('multi_bAllos_1_allos','multi_bAllos_2_allos')){
            
        early_dat_i$Barcode <- paste0(sample_specs$bc_extension[sample_specs$sample == earlyAllo_dataset],'allos_',early_dat_i$Barcode,'-1')
        
    }else{
            
        early_dat_i$Barcode <- paste0(sample_specs$bc_extension[sample_specs$sample == earlyAllo_dataset],early_dat_i$Barcode)
            
    }
    
    
    if(!is.na(pt_allo_combos[i,'lateAllo_dataset'])){
        
        late_dat_i <- read.delim(paste0(sample_specs_pt$sample_wd[i],'/unfiltered_comb_seq_ids_with_PT_assignment_lateAllos.csv'), sep = ',', stringsAsFactors = F, row.names = 1)
        late_dat_i <- late_dat_i[!duplicated(late_dat_i$bc_clone),]
        late_dat_i <- late_dat_i[,c('Barcode','code_combinations','pt_fish','fish_clone','inferred_clone','assigned_to_pt_fish','assigned_to_pt_clone')]
        late_dat_i$dataset_allo <- lateAllo_dataset

        if(lateAllo_dataset == 'multi_bAllos_2_lateAllos'){
            late_dat_i$Barcode <- paste0(sample_specs$bc_extension[sample_specs$sample == lateAllo_dataset],'lateAllos_',late_dat_i$Barcode,'-1')
        }else{
            late_dat_i$Barcode <- paste0(sample_specs$bc_extension[sample_specs$sample == lateAllo_dataset],late_dat_i$Barcode)
        }
        
        early_dat_i <- rbind(early_dat_i, late_dat_i)
        rm(late_dat_i)

    }
    
    allClones <- rbind(allClones, early_dat_i)

}

[1] "Adding dataset multi_seq_06"
[1] "Adding dataset multi_seq_07"
[1] "Adding dataset multi_seq_08"
[1] "Adding dataset multi_seq_09"


In [128]:
write.csv(allClones, './data/allAssignedClones_inGraftData_merged.csv', quote = F)


### Load clone assignments and merge with metadata

In [7]:
# Load info from allografts
clones_in_allos <- read.delim('./data/allAssignedClones_inGraftData_merged.csv', stringsAsFactors = F, sep = ',', row.names = 1)
#clones_in_allos <- read.delim('/local/users/nfresma/R_out/all_clones/allAssignedClones_inGraftData_merged.csv', stringsAsFactors = F, sep = ',', row.names = 1)

# Load info from primary tumors
clones_in_pts <- read.delim('./data/metadata_with_clones_merged_PTs_NBcells_lowRes.csv',stringsAsFactors = F, sep = ',', row.names = 1)
rownames(clones_in_pts) <- paste0('pts_',rownames(clones_in_pts))


In [8]:
head(clones_in_pts)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,names,manual_calls,negcell_reclass_calls,dombc_calls,dombc2_calls,cons_class,⋯,celltype_det,celltype_overall,celltype_group,celltype_doub,Barcode,fish_clone,inferred_clone,common_cluster,dataset.y,BC_cloneID
,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
pts_multi_2_multi_seq_2_s1_AAACCCAGTCGGCCTA-1,multi_seq_2_s1,2879,663,0.6252171,AAACCCAGTCGGCCTA-1,Bar5,Negative,Bar5,Negative,Bar5,⋯,NB_ery_contaminated,NB,NB,NB_ery_contaminated,multi_2_multi_seq_2_s1_AAACCCAGTCGGCCTA-1,NA,NA,NA,NA,NA
pts_multi_2_multi_seq_2_s1_AAACCCAGTGTGGTCC-1,multi_seq_2_s1,11594,2145,4.2694497,AAACCCAGTGTGGTCC-1,Bar15,Bar15,Doublet,Bar15,Bar15,⋯,NB,NB,NB,NB,multi_2_multi_seq_2_s1_AAACCCAGTGTGGTCC-1,NA,NA,NA,NA,NA
pts_multi_2_multi_seq_2_s1_AAACCCAGTTCTCAGA-1,multi_seq_2_s1,2282,613,8.5013146,AAACCCAGTTCTCAGA-1,Bar11,Bar11,Doublet,Bar11,Bar11,⋯,NB,NB,NB,NB,multi_2_multi_seq_2_s1_AAACCCAGTTCTCAGA-1,NA,NA,NA,NA,NA
pts_multi_2_multi_seq_2_s1_AAACCCATCAGCGTCG-1,multi_seq_2_s1,3593,685,1.4472586,AAACCCATCAGCGTCG-1,Bar15,Bar15,Bar15,Negative,Bar15,⋯,NB_ery_contaminated,NB,NB,NB_ery_contaminated,multi_2_multi_seq_2_s1_AAACCCATCAGCGTCG-1,NA,NA,NA,NA,NA
pts_multi_2_multi_seq_2_s1_AAACCCATCAGCTGTA-1,multi_seq_2_s1,12254,1868,3.4192917,AAACCCATCAGCTGTA-1,Bar6,Bar6,Bar6,Bar6,Bar6,⋯,NB,NB,NB,NB,multi_2_multi_seq_2_s1_AAACCCATCAGCTGTA-1,NA,NA,NA,NA,NA
pts_multi_2_multi_seq_2_s1_AAACGAACAAAGTGTA-1,multi_seq_2_s1,5396,1386,3.9103039,AAACGAACAAAGTGTA-1,Bar9,Bar9,Doublet,Bar9,Bar9,⋯,NB,NB,NB,NB,multi_2_multi_seq_2_s1_AAACGAACAAAGTGTA-1,NA,NA,NA,NA,NA


In [130]:
# Merge allograft clone info with metadata
metadat_ext <- left_join(dat@meta.data, clones_in_allos, by = 'Barcode')
rownames(metadat_ext) <- metadat_ext$Barcode

table(rownames(metadat_ext) == rownames(dat@meta.data))


 TRUE 
88626 

In [131]:
# If above row names match, add to Seurat object
dat@meta.data <- metadat_ext

In [132]:
# Merge datasets from one allograft experiment
dat@meta.data$experiment <- as.character(dat@meta.data$dataset)
dat@meta.data$experiment[dat@meta.data$dataset %in% c('multi_seq_06','multi_seq_12')] <- 'multi_seq_06'
dat@meta.data$experiment[dat@meta.data$dataset %in% c('multi_seq_07','multi_seq_13','multi_seq_14')] <- 'multi_seq_07'
dat@meta.data$experiment[dat@meta.data$dataset %in% c('multi_seq_08','multi_seq_15')] <- 'multi_seq_08'
dat@meta.data$experiment[dat@meta.data$dataset %in% c('multi_seq_09','multi_seq_16','multi_seq_18','multi_seq_19')] <- 'multi_seq_09'

# Merge clone and fish info with dataset
dat@meta.data$dataset_pt_fish <- paste0(dat@meta.data$experiment, '_', as.character(dat@meta.data$pt_fish))
dat@meta.data$dataset_pt_fish[is.na(dat@meta.data$pt_fish)] <- NA
dat@meta.data$dataset_pt_clone <- paste0(dat@meta.data$experiment, '_', as.character(dat@meta.data$inferred_clone))
dat@meta.data$dataset_pt_clone[is.na(dat@meta.data$inferred_clone)] <- NA

# Rename columns to clearly label allograft cells and to avoid clashes with column names from primary tumor data
dat@meta.data$pt_inferred_clone <- dat@meta.data$inferred_clone
dat@meta.data$pt_fish_clone <- dat@meta.data$fish_clone


In [134]:
dat@meta.data$inferred_clone <- NULL
dat@meta.data$fish_clone <- NULL

In [135]:
# Merge PT clone info with metadata
clones_in_pts_rem <- clones_in_pts[!rownames(clones_in_pts) %in% rownames(dat@meta.data),]
clones_in_pts <- clones_in_pts[rownames(clones_in_pts) %in% rownames(dat@meta.data),]

clones_in_pts$fish_clone <- as.character(clones_in_pts$fish_clone)
clones_in_pts$Barcode <- rownames(clones_in_pts)
clones_in_pts <- clones_in_pts[,c('Barcode','fish_clone','inferred_clone')]

table(clones_in_pts$Barcode %in% dat@meta.data$Barcode)

clones_in_pts_ext <- left_join(dat@meta.data, clones_in_pts, by = 'Barcode')
rownames(clones_in_pts_ext) <- clones_in_pts_ext$Barcode

table(rownames(clones_in_pts_ext) == rownames(dat@meta.data))


 TRUE 
30184 


 TRUE 
88626 

In [136]:
# If above row names match, add to Seurat object
dat@meta.data <- clones_in_pts_ext

In [139]:
# Shorten clone names by removing duplicate fish_of_origin info.
dat@meta.data$fish_clone_pt_short <- gsub("_[0-9]+_clone", "", dat@meta.data$inferred_clone)
dat@meta.data$fish_clone_pt_short <- gsub('fish_','',dat@meta.data$fish_clone_pt_short)

dat@meta.data$pt_fish_clone[dat@meta.data$pt_fish_clone %like any% '%_NA'] <- NA
dat@meta.data$fish_clone_pt_short[dat@meta.data$fish_clone_pt_short %like any% '%_NA'] <- NA

# Merge information on clones for all timepoints into one column
dat@meta.data$clone_all_tps <- dat@meta.data$dataset_pt_clone
dat@meta.data$clone_all_tps[dat@meta.data$clone_all_tps == 'no'] <- NA
dat@meta.data$clone_all_tps[dat@meta.data$sample_type == 'primary_tumor'] <- dat@meta.data$fish_clone_pt_short[dat@meta.data$sample_type == 'primary_tumor'] 

# Merge information on assigned fish of origin for all timepoints into one column
dat@meta.data$fish_assigned_all_tps <- dat@meta.data$pt_fish
dat@meta.data$fish_assigned_all_tps[dat@meta.data$sample_type == 'primary_tumor'] <- dat@meta.data$fish_id[dat@meta.data$sample_type == 'primary_tumor']
dat@meta.data$fish_assigned_all_tps <- paste0(dat@meta.data$experiment,'_',dat@meta.data$fish_assigned_all_tps)
dat@meta.data$fish_assigned_all_tps[dat@meta.data$fish_assigned_all_tps %like any% '%_NA'] <- NA

# Add a shortened sample type identifier and merge with fish info
dat@meta.data$sample_type_short <- dat@meta.data$sample_type
dat@meta.data$sample_type_short[dat@meta.data$sample_type_short == 'primary_tumor'] <- 'PTs'
dat@meta.data$sample_type_short[dat@meta.data$sample_type_short == 'early_allograft'] <- 'eA'
dat@meta.data$sample_type_short[dat@meta.data$sample_type_short == 'late_allograft'] <- 'lA'

dat@meta.data$tp_fish <- paste0(dat@meta.data$fish_assigned_all_tps, '_',dat@meta.data$sample_type_short)
dat@meta.data$tp_fish[dat@meta.data$tp_fish %like any% 'NA%'] <- 'NA'


In [145]:
# Save metadatadat_name
write.csv(dat@meta.data, './data/metadata_with_clones_allTPs.csv', quote = F)
write.csv(dat@meta.data[dat@meta.data$ct_final == 'NB',], './data/metadata_with_clones_allTPs_NBcells.csv', quote = F)


## Load module scores and merge with clone info and other metadata
### For zebrafish NB modules

In [146]:
# Load final module table
modules_pan <- read.delim('../gene_modules/final_gene_modules_list.csv', sep = ',', stringsAsFactors = F, header = T)

# Re-format into list
modules_pan <- as.list(modules_pan)

# Double-check that common gene names contain correct set of special characters
modules_pan <- lapply(modules_pan, function(x) gsub("si\\.","si:",x))
modules_pan <- lapply(modules_pan, function(x) gsub("zgc\\.","zgc:",x))
modules_pan <- lapply(modules_pan, function(x) gsub("mt\\.","mt-",x))
modules_pan <- lapply(modules_pan, function(x) gsub("dkey\\.","dkey-",x))
modules_pan <- lapply(modules_pan, function(x) gsub("h211\\.","h211-",x))
modules_pan <- lapply(modules_pan, function(x) gsub("1073\\.","1073-",x))
modules_pan <- lapply(modules_pan, function(x) gsub("1073\\.","1073-",x))
modules_pan <- lapply(modules_pan, function(x) gsub("ch73\\.","ch73-",x))

# Remove empty entries
modules_pan <- lapply(modules_pan, function(x) {
  # Remove NA and empty string "" entries
  x[!(is.na(x) | x == "")]
})

# Remove modules that are too short
modules_pan <- modules_pan[lengths(modules_pan) > 5]

In [147]:
# Load Seurat metadata with clone assignments

metadat_ext <- read.delim('./data/metadata_with_clones_allTPs_NBcells.csv', stringsAsFactors = F, sep = ',', row.names = 1, header = T)
metadat_ext$Barcode <- rownames(metadat_ext)


### Load AUCell module expression scores

In [149]:
load('../transcriptome_analysis/data/moduleScores_AUC_NBcells_final_modules_allTPs_merged_perSample.Robj')

In [154]:
# Merge all scores from different datasets into one dataframe
indtumours <- enrich_list

indtumours_merge <- data.frame()

for(i in 1:length(indtumours)){

    if(is.matrix(indtumours[[i]]) == T){
        indtumours_merge <- rbind(indtumours_merge, indtumours[[i]])
    }else{
        next
    }
}

indtumours_merge <- indtumours_merge[,sort(colnames(indtumours_merge))]


In [156]:
metadat_scores_auc <- indtumours_merge

rm(indtumours_merge)
dim(metadat_scores_auc)

[1] 131774     47

### Load 0-1 module expression scores

In [ ]:
load('../transcriptome_analysis/data/moduleScores_Barkley_NBcells_final_modules_allTPs_merged_perSample.Robj')

In [158]:
# Merge all scores from different datasets into one dataframe
indtumours <- enrich_list

indtumours_merge <- data.frame()

for(i in 1:length(indtumours)){

    if(is.matrix(indtumours[[i]]) == T){
        indtumours[[i]] <- t(indtumours[[i]])
        indtumours_merge <- rbind(indtumours_merge, indtumours[[i]])
    }else{
        next
    }
}

rm(indtumours)

In [160]:
metadat_scores_bark <- indtumours_merge

rm(indtumours_merge)
dim(metadat_scores_bark)

[1] 131630     47

### Load countsums style module scores

In [ ]:
load('../transcriptome_analysis/data/moduleScores_CountSum_NBcells_final_modules_allTPs_merged_perSample.Robj')

In [162]:
# Merge all scores from different datasets into one dataframe
indtumours <- enrich_list

indtumours_merge <- data.frame()

for(i in 1:length(indtumours)){

    if(is.data.frame(indtumours[[i]]) == T){
        indtumours_merge <- rbind(indtumours_merge, indtumours[[i]])
    }else{
        next
    }
}

indtumours_merge <- indtumours_merge[,sort(colnames(indtumours_merge))]


rm(indtumours)

In [164]:
metadat_scores_counts <- indtumours_merge

rm(indtumours_merge)
dim(metadat_scores_counts)

[1] 131630     47

### Harmonize all scores and add them to metadata and as dimensionality reductions

In [165]:
metadat_scores_bark <- metadat_scores_bark[,colnames(metadat_scores_auc)]
table(colnames(metadat_scores_bark) == colnames(metadat_scores_auc))
table(colnames(metadat_scores_bark) == colnames(metadat_scores_counts))
table(colnames(metadat_scores_auc) == colnames(metadat_scores_counts))


TRUE 
  47 


TRUE 
  47 


TRUE 
  47 

In [166]:
metadat_scores_bark_met <- metadat_scores_bark
colnames(metadat_scores_bark_met) <- paste0('barkley_',colnames(metadat_scores_bark_met))
metadat_scores_bark_met$Barcode <- rownames(metadat_scores_bark_met)
metadat_scores_bark_met$barkley_Barcode <- NULL

metadat_scores_auc_met <- metadat_scores_auc
colnames(metadat_scores_auc_met) <- paste0('auc_',colnames(metadat_scores_auc_met))
metadat_scores_auc_met$Barcode <- rownames(metadat_scores_auc_met)
metadat_scores_auc_met$auc_Barcode <- NULL

metadat_scores_counts_met <- metadat_scores_counts
colnames(metadat_scores_counts_met) <- paste0('counts',colnames(metadat_scores_counts_met))
metadat_scores_counts_met$Barcode <- rownames(metadat_scores_counts_met)
metadat_scores_counts_met$counts_Barcode <- NULL

In [167]:
# Merge all with metadata
metadat_ext_ext <- inner_join(metadat_ext, metadat_scores_bark_met, by = 'Barcode')
dim(metadat_ext_ext)
metadat_ext_ext <- left_join(metadat_ext_ext, metadat_scores_auc_met, by = 'Barcode')
dim(metadat_ext_ext)
metadat_ext_ext <- left_join(metadat_ext_ext, metadat_scores_counts_met, by = 'Barcode')
dim(metadat_ext_ext)

rownames(metadat_ext_ext) <- metadat_ext_ext$Barcode

[1] 57601    95

[1] 57601   142

[1] 57601   189

In [168]:
# prevent column duplication
metadat_ext_ext <- metadat_ext_ext[,!colnames(metadat_ext_ext) %in% colnames(dat@meta.data)]
table(colnames(metadat_ext_ext) %in% colnames(dat@meta.data))



FALSE 
  141 

In [169]:
# prepare for merging
metadat_ext_ext$Barcode <- rownames(metadat_ext_ext)
dat@meta.data$Barcode <- rownames(dat@meta.data)

In [170]:
# merge scores with metadata
metadat_all <- left_join(dat@meta.data, metadat_ext_ext, by = 'Barcode')
rownames(metadat_all) <- metadat_all$Barcode
table(rownames(metadat_all) == rownames(dat@meta.data))


 TRUE 
88626 

In [171]:
# If the above row names match, add to Seurat object
dat@meta.data <- metadat_all

In [172]:
# Only keep cells that have a module score and belong to the selected datasets
dim(dat)
dat <- subset(dat, cells = rownames(metadat_ext_ext))
dim(dat)

[1] 26338 88626

[1] 26338 57601

#### Add Scores as dimensionality reduction objects to Seurat object

In [173]:
module_names <- colnames(metadat_scores_auc)
module_names <- module_names[module_names != 'Barcode']

metadat_scores_bark <- metadat_scores_bark[rownames(metadat_scores_bark) %in% rownames(dat@meta.data),]
metadat_scores_bark$Barcode <- NULL
nmf_obj_bark <- as.matrix(metadat_scores_bark)
colnames(nmf_obj_bark) <- paste0('bark', c(1:ncol(metadat_scores_bark)))

metadat_scores_auc <- metadat_scores_auc[rownames(metadat_scores_auc) %in% rownames(dat@meta.data),]
metadat_scores_auc$Barcode <- NULL
nmf_obj_auc <- as.matrix(metadat_scores_auc)
colnames(nmf_obj_auc) <- paste0('auc', c(1:ncol(metadat_scores_auc)))

metadat_scores_counts <- metadat_scores_counts[rownames(metadat_scores_counts) %in% rownames(dat@meta.data),]
metadat_scores_counts$Barcode <- NULL
nmf_obj_counts <- as.matrix(metadat_scores_counts)
colnames(nmf_obj_counts) <- paste0('count', c(1:ncol(metadat_scores_counts)))

In [174]:
nmf_obj_bark <- nmf_obj_bark[rownames(nmf_obj_bark) %in% rownames(dat@meta.data),]
nmf_obj_auc <- nmf_obj_auc[rownames(nmf_obj_auc) %in% rownames(dat@meta.data),]
nmf_obj_counts <- nmf_obj_counts[rownames(nmf_obj_counts) %in% rownames(dat@meta.data),]

In [175]:
# Reorder cell names to match Seurat object
nmf_obj_bark <- nmf_obj_bark[dat$Barcode,]
nmf_obj_auc <- nmf_obj_auc[dat$Barcode,]
nmf_obj_counts <- nmf_obj_counts[dat$Barcode,]

In [176]:
table(rownames(nmf_obj_bark) == dat$Barcode)
table(rownames(nmf_obj_auc) == dat$Barcode)
table(rownames(nmf_obj_counts) == dat$Barcode)


 TRUE 
57601 


 TRUE 
57601 


 TRUE 
57601 

In [177]:
# Add as Dim Red to Seurat object
dat[["scores_bark"]] <- CreateDimReducObject(embeddings = nmf_obj_bark, key = "bark_", assay = DefaultAssay(dat))
dat[["scores_auc"]] <- CreateDimReducObject(embeddings = nmf_obj_auc, key = "auc_", assay = DefaultAssay(dat))
dat[["scores_count"]] <- CreateDimReducObject(embeddings = nmf_obj_counts, key = "count_", assay = DefaultAssay(dat))

In [179]:
# Save to file
saveRDS(dat, file = './data/seur_obj_allmerged_final_NB_with_clones_and_scores_withLowResClones.rds')

In [178]:
# Save order of module names
write.csv(colnames(metadat_scores_bark), './data/mod_name_order_in_data_allTPs.csv', quote = F)